# Share components, environments and models across workspaces

This is the companion notebook for the article on sharing components, environments and models across workspaces: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries 

### Prerequisites
Review the prerequisites section in the article: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries?tabs=python#prerequisites. To summarize, in addition to the Python SDK, you need an AzureML registry and an AzureML workspace in a region that is supported by the workspace.


### Scenarios

There are two scenarios where you'd want to use the same set of models, components and environments in multiple workspaces:

* Cross-workspace MLOps: You're training a model in a dev workspace and need to deploy it to test and prod workspaces. In this case you, want to have end-to-end lineage between endpoints to which the model is deployed in test or prod workspaces and the training job, metrics, code, data and environment that was used to train the model in the dev workspace.
* Share and reuse models and pipelines across different teams: Sharing and reuse improve collaboration and productivity. In this scenario, you may want to publish a trained model and the associated components and environments used to train it to a central catalog. From there, colleagues from other teams can search and reuse the assets you shared in their own experiments.

### Goals
* Create an environment and component in the registry.
* Use the component from registry to submit a model training job in a workspace.
* Register the trained model in the registry.
* Deploy the model from the registry to an online-endpoint in the workspace, then submit an inference request.


In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import (
    Environment,
    BuildContext,
    Model,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes
import time, datetime, os
from dotenv import load_dotenv

load_dotenv(override=True)

SUBSCRIPTION_ID = os.environ["SUBSCRIPTION_ID"]
RESOURCE_GROUP = os.environ["RESOURCE_GROUP"]
AML_WORKSPACE_NAME = os.environ["AML_WORKSPACE_NAME"]
EXTERNAL_REGISTRY_NAME = "publicregistry"#os.environ["EXTERNAL_REGISTRY_NAME"]
USER_ASSIGNED_IDENTITY_NAME = os.environ["USER_ASSIGNED_IDENTITY_NAME"]
USER_ASSIGNED_IDENTITY_RESOURCE_GROUP = os.environ["USER_ASSIGNED_IDENTITY_RESOURCE_GROUP"]
USER_ASSIGNED_IDENTITY_CLIENT_ID = os.environ["USER_ASSIGNED_IDENTITY_CLIENT_ID"]
# print the sdk version - you many want to share this in the issue you will report if parts of this notebook don't work
#!pip show azure-ai-ml

### Setup authentication

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## Connect to a workspace and registry

Most samples create one client to connect to the workspace. However, in this sample, you need two clients. First client, called `ml_client_workspace`, will be used to connect to a workspace and run jobs or deploy endpoints. Second client, called `ml_client_registry` will be used to connect to the registry to create components, environments and models.

Replace the following:
* `<SUBSCRIPTION_ID>`
* `<RESOURCE_GROUP>`
* `<AML_WORKSPACE_NAME>`
* `<REGISTRY_NAME>`
 

In [3]:
ml_client_workspace = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=AML_WORKSPACE_NAME,
)
print(ml_client_workspace)

ml_client_registry = MLClient(credential=credential, registry_name=EXTERNAL_REGISTRY_NAME)
print(ml_client_registry)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001BB86752A50>,
         subscription_id=5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25,
         resource_group_name=rg-aml-ws-dev-cc,
         workspace_name=amldevcc002)


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001BB86752A50>,
         subscription_id=5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25,
         resource_group_name=openexternalci,
         workspace_name=None)


### Create a version number and setup root directory 
Make sure that you set the version number to something unique if this notebook has been run before. You can use the timestamp to generate a unique version number, the sample code for which is commented out. This will prevent any name and version conflicts when creating assets.

Set the root directory in which the YAML definitions of the components, environments, etc. are present.

In [4]:
import time
import sys

# version = str(123456)
version = str(int(time.time()))
print("version: ", version)

version:  1753806783


### Create model in registry

You will now obtain the model trained by the pipeline job in the above step and create the model in the registry. For completeness, we are showing two options here.
* First option shows to create a model in registry from job output without downloading it. This option is recommended when you want to track the lineage between the training job and the model. You will create a model directly from the job output (without downloading it) in your workspace and then copy the model from workspace to registry. 
* Second option shows how to create a model in registry from local files. In this case you will download the model from the job output. This option is helpful if you have an existing model from some external source and want to host it in the registry to be shared with many workspaces.

Review this notebook to learn the different model types and how to create them in a workspace: [../../assets/model/model.ipynb](../model/model.ipynb). In the below example, you will work with a `mlflow_model` that will help you deploy this model for inference without writing any scoring scripts.


### Create model from local files 
Step b: Create a model in registry from files in a local folder. Note that you use the `ml_client_registry` client to create the model in registry. The syntax for creating model in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

> **Warning:** If you have successfully created a model in registry in the previous steps, this section will fail as the model with the name and version will already exist. 

In [5]:
# this section is optional, will fail if this model name and version is already created in the registry in the previous steps
mlflow_model = Model(
    path="./artifacts/model/",
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model",
    version= version,
    description="MLflow model created from local path",
)
print(mlflow_model)
#ml_client_workspace.models.create_or_update(mlflow_model)
ml_client_registry.models.create_or_update(mlflow_model)

description: MLflow model created from local path
name: nyc-taxi-model
path: C:\Users\jomedin\Documents\MLOPs-AzureML\notebooks\asset_sharing\artifacts\model
properties: {}
tags: {}
type: mlflow_model
version: '1753806783'



Subtype value SAS has no mapping, use base class DataReferenceCredentialDto.
Uploading model (0.0 MBs): 100%|##########| 2666/2666 [00:00<00:00, 31753.56it/s]




Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': 'azureml://registries/publicregistry/models/nyc-taxi-model/versions/1753806783', 'Resource__source_path': '', 'base_path': 'c:\\Users\\jomedin\\Documents\\MLOPs-AzureML\\notebooks\\asset_sharing', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000001BB887B07D0>, 'serialize': <msrest.serialization.Serializer object at 0x000001BB88797620>, 'version': '1753806783', 'latest_version': None, 'path': 'https://74d982b71cb.blob.core.windows.net/publicregi-cb264f95-f8b9-5ea5-bc0a-3aa94e900fc6/model', 'datastore': None, 'utc_time_created': None, 'flavors': {'python_function': {'env': 'conda.yaml', 'loader_module': 'mlflow.sklearn', 'model_path': 'model.pkl', 'predi

### Create model in workspace

In [8]:
# this section is optional, will fail if this model name and version is already created in the registry in the previous steps
mlflow_model = Model(
    path="./artifacts/model/",
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model",
    version= version,
    description="MLflow model created from local path",
)
print(mlflow_model)
ml_client_workspace.models.create_or_update(mlflow_model)

description: MLflow model created from local path
name: nyc-taxi-model
path: /mnt/batch/tasks/shared/LS_root/mounts/clusters/azuremlci/code/Users/admin/external_registry/artifacts/model
properties: {}
tags: {}
type: mlflow_model
version: '1753488311'



Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/PublicAzureML/providers/Microsoft.MachineLearningServices/workspaces/pazuremlregistrytest/models/nyc-taxi-model/versions/1753488311', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/azuremlci/code/Users/admin/external_registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x75063e99fe10>, 'serialize': <msrest.serialization.Serializer object at 0x75063e957150>, 'version': '1753488311', 'latest_version': None, 'path': 'azureml://subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/PublicAzureML/workspaces/pazuremlregistrytest/datasto

### Share model from workspace to registry

In [9]:
ml_client_workspace.models.share(
    name="nyc-taxi-model",
    version=version,
    registry_name=EXTERNAL_REGISTRY_NAME,
    share_with_name="nyc-taxi-model-new",
    share_with_version=version,
)

Method share: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nyc-taxi-model-new', 'description': 'MLflow model created from local path', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': 'azureml://registries/publicregistry/models/nyc-taxi-model-new/versions/1753488311', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/azuremlci/code/Users/admin/external_registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x75063d55c640>, 'serialize': <msrest.serialization.Serializer object at 0x75063dffcbb0>, 'version': '1753488311', 'latest_version': None, 'path': 'https://74d982b71cb.blob.core.windows.net/publicregi-6bff3d61-5e25-5de4-b0a1-a26950d059af/LocalUpload/beaf4dff7bd507b2efc4d854506d9a608734143a0118f0b24568f96a7a57852a/model', 'datastore': None, 'utc_time_created': None, 'flavors': {'python_fu

## Create environment in workspace and then share it to registry

### Create environment in workspace

In [11]:
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="environment/online-endpoints-multimodel.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client_workspace.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourceGroups/PublicAzureML/providers/Microsoft.MachineLearningServices/workspaces/pazuremlregistrytest/environments/docker-image-plus-conda-example/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/azuremlci/code/Users/admin/external_registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x75063f37a0f0>, 'serialize': <msrest.serialization.Serializer object at 0x75063e9567b0>, 'version': '1', 'c

### Share environment to registry

In [ ]:
ml_client_workspace.environments.share(
    name="docker-image-plus-conda-example",
    version=version,
    registry_name=EXTERNAL_REGISTRY_NAME,
    share_with_name="docker-image-plus-conda-example",
    share_with_version=version,
)

### Deploy model from registry to online endpoint in workspace

You will deploy the model to an online endpoint and submit some sample inference requests in this section. Note that just like jobs, endpoints that host models are specific to a workspace. You can deploy the a model from a registry to many workspaces. This helps you develop a model in `dev` workspace, share it with a registry, and then deploy it to `test` and `prod` workspaces. 

### Get model from registry

Use the `ml_client_registry` client to get the model created in previous section from the registry. The syntax for creating component in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

In [16]:
mlflow_model_from_registry = ml_client_registry.models.get(
    name="nyc-taxi-model", version=version
)
print(mlflow_model_from_registry)

creation_context:
  created_at: '2025-07-17T04:11:36.075279+00:00'
  created_by: 7eb0cccd-2e21-4ca5-abb7-c65071ac0203
  created_by_type: Application
  last_modified_at: '2025-07-17T04:11:35.571473+00:00'
  last_modified_by: 7eb0cccd-2e21-4ca5-abb7-c65071ac0203
  last_modified_by_type: Application
description: MLflow model created from local path
flavors:
  python_function:
    env: conda.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.8.13
  sklearn:
    code: ''
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 0.24.1
id: azureml://registries/publicregistry/models/nyc-taxi-model/versions/1752725482
name: nyc-taxi-model
path: https://74d982b71cb.blob.core.windows.net/publicregi-1fd23ebd-f2b7-51d7-be28-0c7b5183df5d/model
properties: {}
tags: {}
type: mlflow_model
version: '1752725482'



### Create an online endpoint 

Create Identity configuration for User Assigned Identity

In [17]:
# https://github.com/MicrosoftDocs/azure-ai-docs/blob/main/articles/machine-learning/how-to-identity-based-service-authentication.md
# https://learn.microsoft.com/en-us/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2&tabs=python#create-compute-with-managed-identity-to-access-docker-images-for-training

In [18]:
from azure.ai.ml.entities import IdentityConfiguration, ManagedIdentityConfiguration
from azure.ai.ml.constants import ManagedServiceIdentityType

uai = USER_ASSIGNED_IDENTITY_NAME  # replace with your user-assigned managed identity name
resource_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourcegroups/{USER_ASSIGNED_IDENTITY_RESOURCE_GROUP}/providers/Microsoft.ManagedIdentity/userAssignedIdentities/{uai}"
# Create an identity configuration from the user-assigned managed identity
managed_identity = ManagedIdentityConfiguration(resource_id=resource_id)
identity_config = IdentityConfiguration(type = ManagedServiceIdentityType.USER_ASSIGNED, user_assigned_identities=[managed_identity])

Create an online endpoint to deploy the model

In [19]:
online_endpoint_name = "endpoint-" + version
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    identity=identity_config,
)
ml_client_workspace.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-1752725482.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-1752725482.eastus2.inference.ml.azure.com/swagger.json', 'name': 'endpoint-1752725482', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {}, 'properties': {'createdBy': '7eb0cccd-2e21-4ca5-abb7-c65071ac0203', 'createdAt': '2025-07-17T04:12:08.018487+0000', 'lastModifiedAt': '2025-07-17T04:12:08.018487+0000', 'azureml.onlineendpointid': '/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/resourcegroups/publicazureml/providers/microsoft.machinelearningservices/workspaces/pazuremlregistrytest/onlineendpoints/endpoint-1752725482', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5784b6a5-de3f-4fa4-8b8f-e5bb70ff6b25/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:850d046a-3781-47a9-86d9-5475e26d52

### Deploy the model from registry to the online endpoint

In [ ]:
# create a demo deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=mlflow_model_from_registry.id,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)
ml_client_workspace.online_deployments.begin_create_or_update(demo_deployment).result()

endpoint.traffic = {"demo": 100}
ml_client_workspace.begin_create_or_update(endpoint).result()

Check: endpoint endpoint-1752725482 exists


...............................

### Test the deployment

This section needs a sample request file `scoring-data.json` which is available in the root directory initialized in the beginning of this notebook: [../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/scoring-data.json](../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/scoring-data.json)

In [ ]:
# test the  deployment with some sample data
ml_client_workspace.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="scoring-data.json",
)

'[5.936981839396367, 21.51612674922012, 55.471569384161285]'

### Clean up resources

#### delete online endpoint 

In [ ]:
print(f"online_endpoint_name: {online_endpoint_name}")
ml_client_workspace.online_endpoints.begin_delete(name=online_endpoint_name).result()

online_endpoint_name: endpoint-1752701048
....................

## Check Registry Configuration

First, let's identify your registries and their resource groups to ensure we're working with the correct one.

In [ ]:
# List all ML registries in the subscription to identify your registry
import subprocess
import json

# Get all ML registries in your subscription
try:
    result = subprocess.run([
        'az', 'ml', 'registry', 'list', 
        '--subscription', SUBSCRIPTION_ID,
        '--query', '[].{Name:name,ResourceGroup:resourceGroup,Location:location}',
        '--output', 'json'
    ], capture_output=True, text=True, check=True)
    
    registries = json.loads(result.stdout)
    print("Available ML Registries:")
    for registry in registries:
        print(f"  Name: {registry['Name']}")
        print(f"  Resource Group: {registry['ResourceGroup']}")
        print(f"  Location: {registry['Location']}")
        print("---")
    
    if not registries:
        print("No ML registries found in the subscription.")
        
except subprocess.CalledProcessError as e:
    print(f"Error listing registries: {e.stderr}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Configure Azure AI Enterprise Network Connection Approver Role
# This role is required for creating managed private endpoints between workspace and external registry

# First, let's check our current user identity
try:
    result = subprocess.run([
        'az', 'account', 'show',
        '--query', '{UserPrincipalName:user.name,TenantId:tenantId}',
        '--output', 'json'
    ], capture_output=True, text=True, check=True)
    
    user_info = json.loads(result.stdout)
    print(f"Current User: {user_info['UserPrincipalName']}")
    print(f"Tenant ID: {user_info['TenantId']}")
    
except Exception as e:
    print(f"Error getting user info: {e}")

print("\n" + "="*60)
print("Azure AI Enterprise Network Connection Approver Role Configuration")
print("="*60)

# Define the role definition ID for Azure AI Enterprise Network Connection Approver
AI_ENTERPRISE_NETWORK_APPROVER_ROLE = "b8bd2b84-0c8e-4c3d-acf1-e0dd1e9f6e5e"

print(f"\nRole Definition ID: {AI_ENTERPRISE_NETWORK_APPROVER_ROLE}")
print("This role allows creating managed private endpoints for Azure AI services.")

# Instructions for role assignment
print("""
REQUIRED ROLE ASSIGNMENT:
To enable asset sharing between workspace and external registry via managed private endpoints,
you need to assign the 'Azure AI Enterprise Network Connection Approver' role.

This role should be assigned at the resource group level containing your registry
to allow the workspace managed identity to create private endpoints.

Once you identify your registry's resource group from the list above,
run the following steps:
""")

# Note: We'll add the actual role assignment after identifying the registry
print("\n⚠️  Next: Identify your registry from the list above, then we'll configure the role assignment.")

In [ ]:
# Assign Azure AI Enterprise Network Connection Approver Role
# Update these variables with your registry details from the list above

# TODO: Update these variables based on your registry information
REGISTRY_NAME = "your-registry-name"  # Replace with your registry name
REGISTRY_RESOURCE_GROUP = "your-registry-rg"  # Replace with your registry's resource group

print("Role Assignment Configuration:")
print(f"Registry Name: {REGISTRY_NAME}")
print(f"Registry Resource Group: {REGISTRY_RESOURCE_GROUP}")
print(f"Workspace Resource Group: {RESOURCE_GROUP}")

def assign_network_approver_role(registry_rg, workspace_rg, subscription_id):
    """Assign Azure AI Enterprise Network Connection Approver role"""
    
    # Get the workspace managed identity
    try:
        result = subprocess.run([
            'az', 'ml', 'workspace', 'show',
            '--name', WORKSPACE_NAME,
            '--resource-group', workspace_rg,
            '--subscription', subscription_id,
            '--query', 'identity.principalId',
            '--output', 'tsv'
        ], capture_output=True, text=True, check=True)
        
        principal_id = result.stdout.strip()
        print(f"Workspace Managed Identity Principal ID: {principal_id}")
        
        if not principal_id or principal_id == 'None':
            print("❌ Workspace does not have a managed identity configured!")
            return False
            
    except Exception as e:
        print(f"❌ Error getting workspace managed identity: {e}")
        return False
    
    # Assign the role to the registry resource group
    try:
        scope = f"/subscriptions/{subscription_id}/resourceGroups/{registry_rg}"
        
        result = subprocess.run([
            'az', 'role', 'assignment', 'create',
            '--assignee', principal_id,
            '--role', AI_ENTERPRISE_NETWORK_APPROVER_ROLE,
            '--scope', scope,
            '--output', 'json'
        ], capture_output=True, text=True, check=True)
        
        assignment = json.loads(result.stdout)
        print(f"✅ Successfully assigned Azure AI Enterprise Network Connection Approver role")
        print(f"   Scope: {scope}")
        print(f"   Principal ID: {principal_id}")
        return True
        
    except subprocess.CalledProcessError as e:
        if "role assignment already exists" in e.stderr.lower():
            print("ℹ️  Role assignment already exists")
            return True
        else:
            print(f"❌ Error assigning role: {e.stderr}")
            return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Instructions
print("\n" + "="*50)
print("INSTRUCTIONS:")
print("="*50)
print("1. Update REGISTRY_NAME and REGISTRY_RESOURCE_GROUP variables above")
print("2. Run this cell again to assign the role")
print("3. The role will be assigned to your workspace's managed identity")

# Uncomment the line below after updating the variables
# assign_network_approver_role(REGISTRY_RESOURCE_GROUP, RESOURCE_GROUP, SUBSCRIPTION_ID)

In [ ]:
# Verify Role Assignment and Test Connectivity
def verify_role_assignment(registry_rg, workspace_rg, subscription_id):
    """Verify the Azure AI Enterprise Network Connection Approver role assignment"""
    
    # Get workspace managed identity
    try:
        result = subprocess.run([
            'az', 'ml', 'workspace', 'show',
            '--name', WORKSPACE_NAME,
            '--resource-group', workspace_rg,
            '--subscription', subscription_id,
            '--query', 'identity.principalId',
            '--output', 'tsv'
        ], capture_output=True, text=True, check=True)
        
        principal_id = result.stdout.strip()
        
        # Check role assignments
        scope = f"/subscriptions/{subscription_id}/resourceGroups/{registry_rg}"
        
        result = subprocess.run([
            'az', 'role', 'assignment', 'list',
            '--assignee', principal_id,
            '--scope', scope,
            '--query', f"[?roleDefinitionId=='{AI_ENTERPRISE_NETWORK_APPROVER_ROLE}']",
            '--output', 'json'
        ], capture_output=True, text=True, check=True)
        
        assignments = json.loads(result.stdout)
        
        if assignments:
            print("✅ Azure AI Enterprise Network Connection Approver role is assigned")
            for assignment in assignments:
                print(f"   Assignment ID: {assignment['id']}")
                print(f"   Scope: {assignment['scope']}")
        else:
            print("❌ Azure AI Enterprise Network Connection Approver role is NOT assigned")
            return False
            
        return True
        
    except Exception as e:
        print(f"❌ Error verifying role assignment: {e}")
        return False

def test_registry_connectivity(registry_name, registry_rg, subscription_id):
    """Test connectivity to the registry"""
    
    try:
        # Try to list assets in the registry
        result = subprocess.run([
            'az', 'ml', 'model', 'list',
            '--registry-name', registry_name,
            '--subscription', subscription_id,
            '--query', 'length(@)',
            '--output', 'tsv'
        ], capture_output=True, text=True, check=True)
        
        model_count = result.stdout.strip()
        print(f"✅ Successfully connected to registry '{registry_name}'")
        print(f"   Found {model_count} models in registry")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Cannot connect to registry '{registry_name}': {e.stderr}")
        return False
    except Exception as e:
        print(f"❌ Error testing connectivity: {e}")
        return False

print("Registry Connectivity Verification")
print("="*40)

# Instructions for verification
print("""
After updating the registry variables and assigning the role, run this verification:

1. Verify role assignment
2. Test registry connectivity
3. Check if managed private endpoints can be created

Update the variables below and uncomment the verification calls:
""")

# TODO: Update these with your actual registry details
VERIFY_REGISTRY_NAME = "your-registry-name"
VERIFY_REGISTRY_RG = "your-registry-rg"

# Uncomment these lines after updating the variables above
# print("\\nVerifying role assignment...")
# role_assigned = verify_role_assignment(VERIFY_REGISTRY_RG, RESOURCE_GROUP, SUBSCRIPTION_ID)

# print("\\nTesting registry connectivity...")
# connectivity_ok = test_registry_connectivity(VERIFY_REGISTRY_NAME, VERIFY_REGISTRY_RG, SUBSCRIPTION_ID)

# if role_assigned and connectivity_ok:
#     print("\\n🎉 Configuration completed successfully!")
#     print("You can now proceed with asset sharing between workspace and registry.")
# else:
#     print("\\n⚠️  Configuration incomplete. Please review the errors above.")